##### 

### dev_notebook_

This is your development environment...you may start:
- Exploring your data.
- Testing the ad-hoc methods.
- Designing your pipeline.

In [1]:
import pandas as pd 
import requests as req
import json
import re

#%pip install shapely
#%pip install geopandas
from shapely.geometry import Point
import geopandas as gpd

#### **API Consulados&Embajadas**

In [30]:
#call the API of the EMT de Madrir
url = 'https://datos.madrid.es/egob'
veronica='/catalogo/201000-0-embajadas-consulados.json'
url=url + veronica

res = req.get(url)  
res

<Response [200]>

In [31]:
#json
data=res.json()


In [32]:
data.keys()

dict_keys(['@context', '@graph'])

In [33]:
graph=data['@graph']

In [34]:
#separate latitude and longitude
dp=pd.DataFrame(graph)
location = pd.DataFrame(pd.json_normalize(dp['location']))



In [35]:
#take the info from address
adress= pd.DataFrame(pd.json_normalize(dp['address']))
adress.head()

,locality,postal-code,street-address,district.@id,area.@id
0,MADRID,28010,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...
1,MADRID,28016,CALLE TALAVERA 9,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...
2,MADRID,28002,CALLE AVIADOR LINDBERGH 3,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...
3,MADRID,28001,CALLE GOYA 5 y 7 pasaje,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...
4,MADRID,28046,PASEO CASTELLANA 18 PLANTA 6 Izquierda,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...


In [36]:
#function to find which embassies already have data from nearby bicimad stations
def find_element_with_word(lista, word):
    for element in lista:
        if word in element:
            return element
        else:
            pass
            
    return 'No tiene Bicimad'
    
   

In [37]:
#apply it to all rows
org=[]
tem_list=[]    
for dic in graph:
    listas=dic['organization']['organization-desc'].split('.  ') 
    tem_list.append(listas)
for lista in tem_list:
    e=find_element_with_word(lista,'Bicimad')
    org.append(e)
           
#org


In [38]:
#concatenate the tables to decide which data we want to use
embajadas=pd.concat([dp, adress, location], axis=1).drop(columns=['id','@id','@type','relation','address', 'organization','district.@id','area.@id','location','locality','postal-code'])
embajadas['number']=org
embajadas['type_of_place']='Embajadas y Consulados'
embajadas.head()

,title,street-address,latitude,longitude,number,type_of_place
0,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,Bicimad: Estaciones 8 (plaza de Alonso Martíne...,Embajadas y Consulados
1,Consulado de Belice,CALLE TALAVERA 9,40.453376,-3.680978,"Bicimad: Estaciones 148 (calle Serrano, 210), ...",Embajadas y Consulados
2,Consulado de Bolivia,CALLE AVIADOR LINDBERGH 3,40.442447,-3.679665,"Bicimad: Estación 146 (calle María Francisca, 1).",Embajadas y Consulados
3,Consulado de Brasil,CALLE GOYA 5 y 7 pasaje,40.425716,-3.688868,"Bicimad: Estaciones 93 (calle Goya, 1) y 106 a...",Embajadas y Consulados
4,Consulado de Bélgica,PASEO CASTELLANA 18 PLANTA 6 Izquierda,40.428626,-3.689004,Bicimad: Estaciones 104 (paseo de la Castellan...,Embajadas y Consulados


In [39]:
#separate the stations near each embassy and return a list with just the numbers

celdas = []

for celda in embajadas['number']:

    if ') y ' in celda:
        celda=celda.replace(') y', '), ')   #unify the format to be able to separate the string 
    
    
    if 'a y b' in celda: #number of stations option
        ayb_list=[] 
         
        if 'Bicimad: Estaciones ' in celda:
            num = celda.split('Bicimad: Estaciones')[1].split('),')         
            for e in num:                                
                if 'a y b' in e:
                    nums=re.findall(r'\b(\d+)\s*a\s*y\s*b\b', e)
                    ayb_list.append(f'{nums[0]}a')
                    ayb_list.append(f'{nums[0]}b')
                else: 
                    ayb_list.append(re.findall('\s\d+\s', e)[0])
            celdas.append(ayb_list)
                
        elif 'Bicimad: Estación ' in celda:
            nums=re.findall('\s\d+\s', celda)
            celdas.append([f'{nums[0].strip()}a', f'{nums[0].strip()}b'])
        
    if 'a y b' not in celda:
        if 'Bicimad: Estaciones ' in celda:
            partes = re.findall('\s\d+\s', celda)
            celdas.append(partes)
        elif 'Bicimad: Estación ' in celda:
            parte = re.findall('\s\d+\s', celda)[0]
            celdas.append(parte)
        else:
            celdas.append('0') #where there is no data
#celdas

In [40]:
#add a colunm
embajadas['number']=celdas


In [41]:
embajadas

,title,street-address,latitude,longitude,number,type_of_place
0,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,"[ 8 , 10 , 104 ]",Embajadas y Consulados
1,Consulado de Belice,CALLE TALAVERA 9,40.453376,-3.680978,"[ 148 , 159 ]",Embajadas y Consulados
2,Consulado de Bolivia,CALLE AVIADOR LINDBERGH 3,40.442447,-3.679665,146,Embajadas y Consulados
3,Consulado de Brasil,CALLE GOYA 5 y 7 pasaje,40.425716,-3.688868,"[ 93 , 106a, 106b, 104 , 105 ]",Embajadas y Consulados
4,Consulado de Bélgica,PASEO CASTELLANA 18 PLANTA 6 Izquierda,40.428626,-3.689004,"[ 104 , 105 , 103 ]",Embajadas y Consulados
...,...,...,...,...,...,...
152,Embajada de Yemen,PASEO CASTELLANA 117 PLANTA 8 D,40.457056,-3.690768,"[ 157 , 156 , 238 , 253 ]",Embajadas y Consulados
153,Embajada de la República Democrática del Congo,CALLE DOCTOR FLEMING 45 PLANTA 1 F,40.462675,-3.687881,"[ 255 , 209 ]",Embajadas y Consulados
154,Embajada de la República Popular Democrática d...,CALLE DARIO APARICIO 43,40.472005,-3.770871,0,Embajadas y Consulados
155,Embajada de la República de Corea,CALLE GONZALEZ AMIGO 15,40.466107,-3.666833,0,Embajadas y Consulados


**API ESTACIONES**

In [42]:
#find the password of another file
with open('../contraseñas.txt', 'r') as file:
    # Leer la contraseña
    contraseña = file.read().strip().split('\n')


In [43]:
#connect to the EMT de madrid API to obtain my accessToken
url = 'https://openapi.emtmadrid.es/v1/mobilitylabs/user/login/'

headers = {
    'email': contraseña[0],
    'password': contraseña[1]
    }

response = req.request('GET', url, headers=headers)
response.json()

{'code': '01',
 'description': 'Token extend  into control-cache Data recovered  OK, (lapsed: 199 millsecs)',
 'datetime': '2024-03-16T09:50:27.048475',
 'data': [{'nameApp': 'OPENAPI MobilityLabs',
   'levelApp': 0,
   'updatedAt': '2024-03-05T18:29:05.2530000',
   'userName': 'veronicagrim@hotmail.com',
   'lastUpdate': {'$date': 1710576362193},
   'idUser': '2c4e1717-b9d6-4bcf-b026-38157bb8f83b',
   'priv': 'U',
   'tokenSecExpiration': 86399,
   'email': 'veronicagrim@hotmail.com',
   'tokenDteExpiration': {'$date': 1710666361193},
   'flagAdvise': True,
   'accessToken': 'f80fe3d1-a76b-452b-bc7e-21302f7228da',
   'apiCounter': {'current': 2,
    'dailyUse': 20000,
    'owner': 0,
    'licenceUse': 'Please mention EMT Madrid MobilityLabs as data source. Thank you and enjoy!',
    'aboutUses': 'If you need to extend the daily use of this API, please, register your App in Mobilitylabs and use your own X-ClientId and  passKey instead of generic login (more info in https://mobilitylabs

In [44]:
#get the accessToken
accessToken=response.json()['data'][0]['accessToken']

In [45]:
#connect to the EMT de madrid API to obtain data from all bicimad stations.
url='https://openapi.emtmadrid.es/v1/transport/bicimad/stations/'

headers = {
    'accessToken': accessToken,
    }

response = req.request("GET", url, headers=headers)
estaciones=response.json()

In [46]:
estaciones.keys()

dict_keys(['code', 'description', 'datetime', 'data'])

In [47]:
#build a dataframe 
estaciones=estaciones['data']
estaciones_api=pd.DataFrame(estaciones)


In [48]:
#clean name and separate latitude and longitude
names=[f['name'].split('-')[1].strip() for f in estaciones]
coordenadas=[[f['geometry']['coordinates'] for f in estaciones]]
coordenadas

estaciones_api['name']=names
estaciones_api['geometry']=coordenadas[0]  
estaciones_api=estaciones_api[['name', 'number', 'address','geometry','activate','no_available','total_bases','dock_bikes','free_bases']]
estaciones_api

,name,number,address,geometry,activate,no_available,total_bases,dock_bikes,free_bases
0,Metro Callao,2,"Calle Miguel Moya nº 1,","[-3.70569, 40.4204]",1,1,27,0,0
1,Plaza Conde Suchil,3,"Plaza Conde Surchill, 4 ,","[-3.7072537, 40.4303223]",1,0,19,11,8
2,Fuencarral,5,"Calle Fuencarral nº 106,","[-3.7021354, 40.4285212]",1,0,27,23,4
3,Colegio Arquitectos,6,"Calle Hortaleza nº 63,","[-3.698447, 40.424148]",1,0,19,2,17
4,Hortaleza,7,"Calle Hortaleza nº 75,","[-3.6977715, 40.4251906]",1,0,19,7,12
...,...,...,...,...,...,...,...,...,...
606,Delicias,273,"Calle de las Delicias, 41 ,","[-3.69091831, 40.40381914]",1,0,19,11,8
607,Camilo José Cela,241,"Calle Camilo José Cela, 27,","[-3.66779, 40.4378]",1,0,23,6,17
608,Casa Encendida,49,"Ronda de Atocha nº 34,","[-3.6992774, 40.4060533]",1,0,27,22,5
609,Alejandro González,187,"Alcala-Ventas ,","[-3.6638249, 40.4301326]",1,0,23,18,5


In [49]:
#separate latitude and longitude
def dividir_lista(row):
    return pd.Series(row['geometry'])
estaciones_api[['longitude', 'latitude']] = estaciones_api.apply(dividir_lista, axis=1)
estaciones_api=estaciones_api.drop(columns=['geometry'])

In [50]:
estaciones_api.head(11)

,name,number,address,activate,no_available,total_bases,dock_bikes,free_bases,longitude,latitude
0,Metro Callao,2,"Calle Miguel Moya nº 1,",1,1,27,0,0,-3.705690,40.420400
1,Plaza Conde Suchil,3,"Plaza Conde Surchill, 4 ,",1,0,19,11,8,-3.707254,40.430322
2,Fuencarral,5,"Calle Fuencarral nº 106,",1,0,27,23,4,-3.702135,40.428521
3,Colegio Arquitectos,6,"Calle Hortaleza nº 63,",1,0,19,2,17,-3.698447,40.424148
4,Hortaleza,7,"Calle Hortaleza nº 75,",1,0,19,7,12,-3.697771,40.425191
5,Metro Alonso Martínez,8,"Plaza de Alonso Martínez nº 6,",1,0,24,14,10,-3.695440,40.427868
6,Plaza de San Miguel,9,"Plaza de San Miguel nº 10,",1,0,24,11,13,-3.709508,40.415606
7,Marqués de la Ensenada,10,"Calle Marqués de la Ensenada (junto a edif, Ce...",1,0,23,14,9,-3.692104,40.425403
8,Plaza del Dos de Mayo,11,"Calle San Andrés nº 18 - 20,",1,0,23,0,23,-3.703600,40.427000
9,San Hermenegildo,12,"Calle San Bernardo nº 85 - 87,",1,0,27,22,5,-3.706141,40.428467


**Embajadas & Estaciones**

In [23]:
#function to measure the distance
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c
def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair points in degrees 
    # (e.g.: Start Point -> 40.4400607 / -3.6425358 End Point -> 40.4234825 / -3.6292625)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

**EMBAJADAS CSV**

In [ ]:
#function to determine the nearest station to each embassy/consulate
def closest_station(embajadas, estaciones_api):
    
    distances = []
    for index, row in estaciones_api.iterrows():
        lat_finish = row['latitude']
        long_finish = row['longitude']
        
        distance = distance_meters(embajadas['latitude'] , embajadas['longitude'], lat_finish, long_finish)
        distances.append([row['number'],distance[0]])
       

    distances.sort(key=lambda x: x[1])

   
    return ([station[0] for station in distances[:3]])

#function to determine the distance between the nearest station to each embassy/consulate   
def closest_distances(embajadas, estaciones_api):
    
    distances = []
    for index, row in estaciones_api.iterrows():
        lat_finish = row['latitude']
        long_finish = row['longitude']
        
        distance = distance_meters(embajadas['latitude'] , embajadas['longitude'], lat_finish, long_finish)
        distances.append([row['number'],distance[0]])
       

    distances.sort(key=lambda x: x[1])


    return ([station[1] for station in distances[:3]])


In [25]:
estaciones_bicimad=[]
for index, row in embajadas.iterrows():
    estaciones_bicimad.append(closest_station(row, estaciones_api))
    
distance=[]
for index, row in embajadas.iterrows():
    distance.append(closest_distances(row, estaciones_api))

NameError: name 'closest_station' is not defined

In [ ]:
distances=[d for e in distance for d in e]


In [ ]:
embajadas['number']=estaciones_bicimad
embajadas=embajadas.explode('number')
embajadas['distance']=distances
embajadas

In [ ]:
for i, row in embajadas.iterrows():
    if 'Consulado' in row['title']:
        embajadas.at[i, 'type_of_place'] = 'consulado'
    else:
        embajadas.at[i, 'type_of_place'] = 'embajada'
        
embajadas

In [ ]:
embajadas.to_csv('../data/processed/embajadas.csv', index=False)

In [ ]:
result = pd.merge(embajadas, estaciones_api, on='number', how='left')
result


**CSV ESTACIONES**

In [24]:
#find the closest station to another bicimad station
def closest_stations_and_distances(row, estaciones_api):
    lat_origen = row['latitude']
    long_origen = row['longitude']
    
    distances = []
    for index, station_row in estaciones_api.iterrows():
        lat_destino = station_row['latitude']
        long_destino = station_row['longitude']
        
        distance = distance_meters(lat_origen, long_origen, lat_destino, long_destino)
        distances.append((station_row['number'], distance[0]))  # Usar tuplas en lugar de diccionarios
        
    distances.sort(key=lambda x: x[1])
    return distances[:2]

closest_stations_distances = []
for index, row in estaciones_api.iterrows():
    closest_stations_distances.append(closest_stations_and_distances(row, estaciones_api))

In [26]:
#add the information to the data frame
actual=[ e[0][0] for e in closest_stations_distances]
cercana=[ e[1][0] for e in closest_stations_distances]
distancia=[ round(e[1][1]) for e in closest_stations_distances]
alternativa=list(zip(actual, cercana))


In [27]:
estaciones_csv=estaciones_api
estaciones_csv.drop(columns=['number','activate', 'no_available', 'total_bases', 'dock_bikes', 'free_bases', 'longitude', 'latitude'], inplace=True)

In [28]:
#add the information to the data frame
estaciones_csv['number']=alternativa
estaciones_csv['distancia']=distancia
estaciones_csv=estaciones_api.explode('number')

estaciones_csv

,name,address,number,distancia
0,Metro Callao,"Calle Miguel Moya nº 1,",2,101
0,Metro Callao,"Calle Miguel Moya nº 1,",22,101
1,Plaza Conde Suchil,"Plaza Conde Surchill, 4 ,",3,241
1,Plaza Conde Suchil,"Plaza Conde Surchill, 4 ,",12,241
2,Fuencarral,"Calle Fuencarral nº 106,",5,42
...,...,...,...,...
608,Casa Encendida,"Ronda de Atocha nº 34,",53,300
609,Alejandro González,"Alcala-Ventas ,",187,324
609,Alejandro González,"Alcala-Ventas ,",193,324
610,"Andaluces, 2","Calle de los Andaluces, 2,",427,344


In [52]:
estaciones.to_csv('../data/processed/estaciones.csv', index=False)

In [51]:
estaciones = pd.merge(estaciones_csv,estaciones_api, on='number', how='left')
estaciones


,name_x,address_x,number,distancia,name_y,address_y,activate,no_available,total_bases,dock_bikes,free_bases,longitude,latitude
0,Metro Callao,"Calle Miguel Moya nº 1,",2,101,Metro Callao,"Calle Miguel Moya nº 1,",1,1,27,0,0,-3.705690,40.420400
1,Metro Callao,"Calle Miguel Moya nº 1,",22,101,Jacometrezo,"Calle Jacometrezo nº 3,",1,1,3,0,0,-3.706538,40.420078
2,Plaza Conde Suchil,"Plaza Conde Surchill, 4 ,",3,241,Plaza Conde Suchil,"Plaza Conde Surchill, 4 ,",1,0,19,11,8,-3.707254,40.430322
3,Plaza Conde Suchil,"Plaza Conde Surchill, 4 ,",12,241,San Hermenegildo,"Calle San Bernardo nº 85 - 87,",1,0,27,22,5,-3.706141,40.428467
4,Fuencarral,"Calle Fuencarral nº 106,",5,42,Fuencarral,"Calle Fuencarral nº 106,",1,0,27,23,4,-3.702135,40.428521
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1217,Casa Encendida,"Ronda de Atocha nº 34,",53,300,Plaza de Lavapiés,"Plazuela de Ana Diosdado,",1,0,24,14,10,-3.700754,40.408309
1218,Alejandro González,"Alcala-Ventas ,",187,324,Alejandro González,"Alcala-Ventas ,",1,0,23,18,5,-3.663825,40.430133
1219,Alejandro González,"Alcala-Ventas ,",193,324,Plaza de las Ventas,"Avenida De los Toreros, 49 ,",1,0,27,19,8,-3.665555,40.432472
1220,"Andaluces, 2","Calle de los Andaluces, 2,",427,344,"Andaluces, 2","Calle de los Andaluces, 2,",1,0,24,20,4,-3.641483,40.391424


**MAP**

In [ ]:
import folium
import requests
import polyline

def obtener_ruta_bicicleta(origen, destino, api_key):
    # URL base de la API de Google Maps Directions
    url = "https://maps.googleapis.com/maps/api/directions/json?"
    
    # Parámetros de la solicitud
    parametros = {
        "origin": origen,
        "destination": destino,
        "mode": "bicycling",  # Modo de transporte: bicicleta
        "key": api_key  # Tu clave de API de Google Maps
    }
    
    # Realizar la solicitud GET a la API de Google Maps Directions
    response = requests.get(url, params=parametros)
    
    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Devolver la respuesta en formato JSON
        return response.json()
    else:
        # Si hay un error en la solicitud, imprimir el mensaje de error
        print(f"Error al obtener la ruta: {response.status_code}")
        return None

# Coordenadas de origen y destino (puedes proporcionar direcciones o coordenadas)
origen = "40.416775,-3.703790"  # Madrid, España
destino = "40.415518,-3.706727"  # Sol, Madrid, España

# Tu clave de API de Google Maps
api_key = ""

# Obtener la ruta en bicicleta
ruta_bicicleta = obtener_ruta_bicicleta(origen, destino, api_key)

# Crear un mapa centrado en la ubicación de origen
mapa = folium.Map(location=[40.416775, -3.703790], zoom_start=14)

# Agregar un marcador en el origen
folium.Marker(location=[40.416775, -3.703790], popup="Origen").add_to(mapa)

# Agregar un marcador en el destino
folium.Marker(location=[40.415518, -3.706727], popup="Destino").add_to(mapa)

# Decodificar los puntos de la ruta
puntos_ruta = ruta_bicicleta["routes"][0]["overview_polyline"]["points"]
coordenadas = polyline.decode(puntos_ruta)

# Agregar la ruta al mapa
folium.PolyLine(locations=coordenadas).add_to(mapa)

# Guardar el mapa como un archivo HTML
mapa.save("ruta_bicicleta.html")

# Mostrar el mapa en el navegador (esto abrirá el navegador predeterminado)
import webbrowser
webbrowser.open("ruta_bicicleta.html")